# TheMovieDB.org
   https://www.themoviedb.org
   
   https://www.themoviedb.org/documentation/api?language=en-US
   
   https://developers.themoviedb.org/3/getting-started/introduction
  
### Request Limits
    40 request every 10 seconds
    https://developers.themoviedb.org/3/getting-started/request-rate-limiting

In [1]:
import os
os.chdir('/home/jovyan/scrgt-bdso-cc-datascience/datapipeline')


FileNotFoundError: [Errno 2] No such file or directory: '/home/jovyan/scrgt-bdso-cc-datascience/datapipeline'

In [2]:
import os
import sys
import time
from datetime import datetime
import urllib.parse
import http.client
import json
import pandas as pd

import logging
logger = logging.getLogger('actorInfo')
logger.setLevel(logging.INFO)


conn = http.client.HTTPSConnection("api.themoviedb.org")
source = 'themoviedb'
payload = "{}"

apikey = 'd1fe56bd9e9956f57bb056a11cdbeb6d'

In [3]:
def searchActor(actor_name):
    urlactor = urllib.parse.quote(actor_name)
    conn.request("GET", "/3/search/person?include_adult=false&page=1&query={0}&language=en-US&api_key={1}".format(urlactor, apikey), payload)

    res = conn.getresponse()
    data = res.read()
    data = json.loads(data)
    for result in data['results']:
        if result['name']== actor_name:
            # return the actor ID
            actorID = result['id']
            return actorID
        else:
            print("No actor found : ", data)

In [4]:
def getActorBio(actorID):
    conn.request("GET", "/3/person/{0}?language=en-US&api_key={1}".format(actorID, apikey), payload)

    res = conn.getresponse()
    data = res.read()
    data = json.loads(data)
    return data

In [5]:
def getActorBiometric(actorId):
    conn.request("GET", "/3/person/{0}/images?api_key={1}".format(actorID, apikey), payload)

    res = conn.getresponse()
    data = res.read()
    data = json.loads(data)
    return data['profiles']


In [6]:
def getActorRoles(actorID):
    conn.request("GET", "/3/person/{0}/movie_credits?language=en-US&api_key={1}".format(actorID, apikey), payload)

    res = conn.getresponse()
    data = res.read()
    data = json.loads(data)
    cast=[]
    for role in data['cast']:
        try:
            release_date=  int( datetime.strptime(role['release_date'].strip(), '%Y-%m-%d').strftime('%Y%m%d'))
            if release_date>=20090601 and release_date <= 20190601:
                cast.append(role)
        except Exception as e:
            logger.error('ERROR parsing release date: '+ str(e))
            
    return cast


In [7]:
def getActorExternalIDs(actorID):
    conn.request("GET", "/3/person/{0}/external_ids?language=en-US&api_key={1}".format(actorID, apikey), payload)

    res = conn.getresponse()
    data = res.read()
    data = json.loads(data)
    return data

In [8]:
def genActorMovieVertex(_actorID, actorFullName, roles):
    actor_movies_vertex =[]
    for movie in roles:
        link = {"actorID" : actorID, 
                "movieID": movie['id'],
                'characterID': movie['credit_id'],
                'character': movie['character'] , 
                'actor': actor,
                'movieTitle': movie['title'],
                '_actorID': _actorID, 
                '_source': source, 
                'actor': actorFullName, 
                'created_utc_dt': datetime.utcnow().strftime("%m/%d/%Y, %H:%M:%S")} 
        actor_movies_vertex.append(link)
    return actor_movies_vertex


In [9]:
topActors = pd.read_csv('./top100Actors.txt', 
                     sep='\t',
                     header=0, 
                    dtype=str)

topActors.head(15)


,Rank,First Name,Last Name,Domestic Box Office,Movies,Average
0,1,Robert,Downey Jr.,"$5,374,149,735",43,"$124,980,226"
1,2,Samuel L.,Jackson,"$5,234,777,328",60,"$87,246,289"
2,3,Scarlett,Johansson,"$4,859,403,632",28,"$173,550,130"
3,4,Tom,Hanks,"$4,692,293,839",51,"$92,005,762"
4,5,Bradley,Cooper,"$4,263,310,067",26,"$163,973,464"
5,6,Tom,Cruise,"$3,937,740,350",42,"$93,755,723"
6,7,Zoe,Saldana,"$3,932,967,185",26,"$151,267,969"
7,8,Chris,Hemsworth,"$3,915,069,280",22,"$177,957,695"
8,9,Chris,Pratt,"$3,866,023,741",17,"$227,413,161"
9,10,Harrison,Ford,"$3,850,910,074",40,"$96,272,752"


In [10]:
actorsDF = topActors[['Rank','First Name','Last Name']]
actorsDF['Full Name']= actorsDF['First Name']+' '+actorsDF['Last Name']
actorsDF = actorsDF.drop(['First Name','Last Name'], axis=1)
actors = actorsDF.values.tolist()

//anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [11]:
import http.client

conn = http.client.HTTPSConnection("api.themoviedb.org")

for actor in actors:
    try:
        _actorID= actor[0]
        actorFullName = actor[1]
        print(actorFullName)
        actorID = searchActor(actorFullName)
        actorBio = getActorBio(actorID)
        actorBio.update({'images': getActorBiometric(actorID)} )
        actorExternalID = getActorExternalIDs(actorID)

        actorCast = getActorRoles(actorID)
        actorBio.update({'cast': actorCast})
        actorBio.update({'_actorID': _actorID, '_source': source, 'created_utc_dt': datetime.utcnow().strftime("%m/%d/%Y, %H:%M:%S")} )

        f = open('./data/person/themoviedb/themoviedb_p_'+_actorID+'.json', 'w+')
        f.write(json.dumps(actorBio))
        f.close()

        actorMovieVertex = genActorMovieVertex( _actorID, actorFullName, actorCast)
        f2 = open('./data/cast/themoviedb/themoviedb_c_'+_actorID+'.json', 'w+')
        f2.write(json.dumps(actorMovieVertex))
        f2.close()
        print('Processed : ', actor)
        time.sleep(15)
    except Exception as e:
        logger.error('ERROR processing actor bio: '+ str(e))


Robert Downey Jr.


ERROR parsing release date: time data '' does not match format '%Y-%m-%d'
ERROR parsing release date: time data '' does not match format '%Y-%m-%d'


Processed :  ['1', 'Robert Downey Jr.']


ERROR parsing release date: time data '' does not match format '%Y-%m-%d'
ERROR parsing release date: time data '' does not match format '%Y-%m-%d'
ERROR parsing release date: time data '' does not match format '%Y-%m-%d'


Samuel L. Jackson
Processed :  ['2', 'Samuel L. Jackson']


...

Chris Pine
Processed :  ['96', 'Chris Pine']
Jon Favreau
Processed :  ['97', 'Jon Favreau']
Nicole Kidman


ERROR parsing release date: time data '' does not match format '%Y-%m-%d'


Processed :  ['98', 'Nicole Kidman']


ERROR parsing release date: time data '' does not match format '%Y-%m-%d'
ERROR parsing release date: time data '' does not match format '%Y-%m-%d'
ERROR parsing release date: time data '' does not match format '%Y-%m-%d'
ERROR parsing release date: time data '' does not match format '%Y-%m-%d'


Simon Pegg
Processed :  ['99', 'Simon Pegg']
Taylor Lautner
Processed :  ['100', 'Taylor Lautner']


# Get Lookup code for genre

In [12]:
conn = http.client.HTTPSConnection("api.themoviedb.org")

conn.request("GET", "/3/genre/movie/list?language=en-US&api_key={0}".format(apikey), payload)
res = conn.getresponse()
data = res.read()
data = json.loads(data)
data

{'genres': [{'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'},
  {'id': 16, 'name': 'Animation'},
  {'id': 35, 'name': 'Comedy'},
  {'id': 80, 'name': 'Crime'},
  {'id': 99, 'name': 'Documentary'},
  {'id': 18, 'name': 'Drama'},
  {'id': 10751, 'name': 'Family'},
  {'id': 14, 'name': 'Fantasy'},
  {'id': 36, 'name': 'History'},
  {'id': 27, 'name': 'Horror'},
  {'id': 10402, 'name': 'Music'},
  {'id': 9648, 'name': 'Mystery'},
  {'id': 10749, 'name': 'Romance'},
  {'id': 878, 'name': 'Science Fiction'},
  {'id': 10770, 'name': 'TV Movie'},
  {'id': 53, 'name': 'Thriller'},
  {'id': 10752, 'name': 'War'},
  {'id': 37, 'name': 'Western'}]}

# Download profile images and label them into a subfolder for each actor


In [13]:
import os
import sys
import time
from datetime import datetime
import urllib.parse
import urllib
import http.client
import json
import pandas as pd
conn = http.client.HTTPSConnection("api.themoviedb.org")

payload = "{}"

for actor in actors:
    try:
        _actorID= actor[0]
        actorFullName = actor[1]

        actorID = searchActor(actorFullName)
        images = getActorBiometric(actorID)
        images
        baseURL='https://image.tmdb.org/t/p/original/'
        prefix='themoviedb_i_'+_actorID+'_'

        if not os.path.exists('data/person/images/'+_actorID):
            os.makedirs('data/person/images/'+_actorID, exist_ok=True)

        for image in images:
            image_name = image['file_path'][1:]
            image_output = prefix+image_name
            f = open('./data/person/images/'+_actorID+'/'+image_output,'wb')
            f.write(urllib.request.urlopen(baseURL+image_name).read())
            f.close()
    except:
        print ('Error : ', actor)